## Calibrate Camera

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
cal_files = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in cal_files:
    cal_img = cv2.imread(fname)
    cal_gray = cv2.cvtColor(cal_img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(cal_gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

In [3]:
cal_img = mpimg.imread(cal_files[0]) # this image is loaded just to get the shape (we assume all images have the same shape)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, cal_img.shape[1::-1], None, None)

# Call cv2.undistort() with theses values
print("Calibration Matrix:\n{} \n\nDistortion Coefficients:\n{}".format(mtx, dist))

Calibration Matrix:
[[1.15777818e+03 0.00000000e+00 6.67113857e+02]
 [0.00000000e+00 1.15282217e+03 3.86124583e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

Distortion Coefficients:
[[-0.24688507 -0.02373155 -0.00109831  0.00035107 -0.00259867]]


## Threshold Functions

In [4]:
# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

In [5]:
# Calculate directional gradient
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    dir = (int(orient == 'x'), int(orient == 'y'))
    
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, *dir, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255 * abs_sobelx / np.max(abs_sobelx))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output

In [6]:
def binary_from_threshold(channel, thresh):
    binary_image = np.zeros_like(channel)
    binary_image[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    
    return binary_image

## Run Lane Detection Pipeline on Videos

In [7]:
from moviepy.editor import VideoFileClip
import imageio
import numpy as np
from PIL import Image
imageio.plugins.ffmpeg.download()
from IPython.display import HTML

In [8]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = [] # [np.array([False])]  
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
        #x values for detected line pixels
        self.allx = None  
        
        #y values for detected line pixels
        self.ally = None

In [9]:
ym_per_pix = 30 / 720 # meters per pixel in y dimension
xm_per_pix = 3.7 / 700 # meters per pixel in x dimension

class LaneDetection():
    def __init__(self, mtx, dist, smooth_frames=10):
        """
        Initialize assuming we've calibrated the camera beforehand
        
        mtx, dist -- Camera calibration parameters
        """
        self.mtx = mtx
        self.dist = dist
        self.M, self.Minv = self._get_perspective_transform_params()
        self.left_line = Line()
        self.right_line = Line()
        self.smooth_frames = smooth_frames
        
    def _get_perspective_transform_params(self):
        src_points = np.float32([[600, 450], [680, 450], [1050, 680], [250, 680]])
        dst_points = np.float32([[300, 0], [900, 0], [900, 720], [300, 720]])
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        Minv = cv2.getPerspectiveTransform(dst_points, src_points)

        return M, Minv
    
    def _warp_image(self, img):
        return cv2.warpPerspective(img, self.M, img.shape[1::-1], flags=cv2.INTER_LINEAR)

    def _unwarp_image(self, img):
        return cv2.warpPerspective(img, self.Minv, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    def _binary_threshold(self, image):
        # X&Y Sobel
        gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(10, 200))
        grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(10, 200))

        # Threshold on RED
        red = image[:,:,0]
        binary_red = binary_from_threshold(red, thresh=(150, 255))

        # Threshold on SATURATION
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        s = hls[:,:,2]
        l = hls[:,:,1]
        binary_lum = binary_from_threshold(l, thresh=(125, 255))
        binary_sat = binary_from_threshold(s, thresh=(90, 255))

        # combine RED, SATURATION, LUMINOSITY, GRADX, GRADY
        combined = np.zeros_like(s)
        combined[(binary_red == 1) & (binary_lum == 1) & ((binary_sat == 1) | ((gradx == 1) & (grady == 1)))] = 1

        mask = np.zeros_like(combined)
        # TODO read these from image
        height = 720
        width = 1280
        region_of_interest_vertices = np.array([[0,height-1], [width/2, int(0.5*height)], [width-1, height-1]], dtype=np.int32)
        cv2.fillPoly(mask, [region_of_interest_vertices], 1)
        masked = cv2.bitwise_and(combined, mask)

        return masked
    
    def _find_lines(self, binary_warped):
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Create an output image to draw on and  visualize the result
        # out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []# Step through the windows one by one

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin

            # Draw the windows on the visualization image
            # cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high), (0,255,0), 2) 
            # cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high), (0,255,0), 2) 

            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        # PLOTTING
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        return left_fit, right_fit, left_fitx, right_fitx, ploty
    
    def _find_lines_next(self, binary_warped):
        left_fit = self.left_line.current_fit[-1]
        right_fit = self.right_line.current_fit[-1]
        
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!    
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
        left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
        left_fit[1]*nonzeroy + left_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
        right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                      ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                      ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        return left_fit, right_fit, left_fitx, right_fitx, ploty
    
    def _draw_lane_overlay(self, image, binary_threshold, curve_radius, pos):
        warp_zero = np.zeros_like(binary_threshold).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([self.left_line.bestx, self.ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_line.bestx, self.ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        cv2.polylines(color_warp, np.int32([pts_left]),False,(0,0,255),thickness=20)
        cv2.polylines(color_warp, np.int32([pts_right]),False,(255,0,),thickness=20)

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        unwarped_img= cv2.warpPerspective(color_warp, self.Minv, (image.shape[1], image.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(image, 1, unwarped_img, 0.3, 0)
        
        cv2.putText(result, "Curvature Radius: {}m".format(round(curve_radius,2)), 
                    org=(20, 50), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=1.33,
                    color=(0, 0, 0))
        
        lr = 'right'
        if pos < 0:
            lr = 'left'
            
        cv2.putText(result, "Vehicle Position: {}m {} of center".format(round(abs(pos), 2), lr),
                    org=(20, 100), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=1.33,
                    color=(0, 0, 0))

        return result
    
    def _curve_radius(self, left_fit, right_fit):
        y_eval = np.max(self.ploty)
        
        left_fit_scaled = np.polyfit(self.ploty * ym_per_pix, self.left_line.bestx * xm_per_pix, 2)
        right_fit_scaled = np.polyfit(self.ploty * ym_per_pix, self.right_line.bestx * xm_per_pix, 2)
        
        left_curverad = ((1 + (2*left_fit_scaled[0]*y_eval + left_fit_scaled[1])**2)**1.5) / np.absolute(2*left_fit_scaled[0])
        right_curverad = ((1 + (2*right_fit_scaled[0]*y_eval + right_fit_scaled[1])**2)**1.5) / np.absolute(2*right_fit_scaled[0])
        
        return (left_curverad + right_curverad) / 2
        
    def _vehicle_pos(self, left_fit, right_fit):
        # TODO Fix these
        center_of_car = 640
        height = 720
        
        left_center = left_fit[0]*height**2 + left_fit[1]*height + left_fit[2]
        right_center = right_fit[0]*height**2 + right_fit[1]*height + right_fit[2]
        lane_center = (left_center + right_center) / 2
        
        return xm_per_pix * (center_of_car - lane_center)
        
    def detect_lane(self, img):
        undistorted_img = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        binary_threshold_img = self._binary_threshold(undistorted_img)
        binary_warped_img = self._warp_image(binary_threshold_img)
        
        if self.left_line.current_fit and self.right_line.current_fit:
            left_fit, right_fit, left_fitx, right_fitx, ploty = self._find_lines_next(binary_warped_img)
        else:
            left_fit, right_fit, left_fitx, right_fitx, ploty = self._find_lines(binary_warped_img)
        
        self.left_line.current_fit.append(left_fit)
        if len(self.left_line.current_fit) >= self.smooth_frames:
            self.left_line.current_fit.pop(0)
            
        self.right_line.current_fit.append(right_fit)
        if len(self.right_line.current_fit) >= self.smooth_frames:
            self.right_line.current_fit.pop(0)
            
        self.left_line.recent_xfitted.append(left_fitx)
        if len(self.left_line.recent_xfitted) >= self.smooth_frames:
            self.left_line.recent_xfitted.pop(0)
        self.left_line.bestx = np.mean(self.left_line.recent_xfitted, axis=0)
                
        self.right_line.recent_xfitted.append(right_fitx)
        if len(self.right_line.recent_xfitted) >= self.smooth_frames:
            self.right_line.recent_xfitted.pop(0)
        self.right_line.bestx = np.mean(self.right_line.recent_xfitted, axis=0)  
        
        self.ploty = ploty
        
        curve_radius = self._curve_radius(left_fit, right_fit)
        vehicle_pos = self._vehicle_pos(left_fit, right_fit)
        
        lane_overlay_img = self._draw_lane_overlay(undistorted_img, binary_threshold_img, curve_radius, vehicle_pos)

        return lane_overlay_img

In [10]:
# provide the path for input and output videos
input_video_path = '../project_video.mp4'
output_video_path = '../output_videos/project_video_output.mp4'

pipeline = LaneDetection(mtx, dist)

In [11]:
clip1 = VideoFileClip(input_video_path)
clip = clip1.fl_image(pipeline.detect_lane)
%time clip.write_videofile(output_video_path, audio=False)

[MoviePy] >>>> Building video ../output_videos/project_video_output.mp4
[MoviePy] Writing video ../output_videos/project_video_output.mp4


100%|█████████▉| 1260/1261 [01:54<00:00, 11.05it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_videos/project_video_output.mp4 

CPU times: user 2min 9s, sys: 26.4 s, total: 2min 35s
Wall time: 1min 54s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video_path))